In [ ]:
from mpcontribs.client import load_client
from pandas import read_excel
from pymatgen import MPRester
import numpy as np
from time import sleep

In [ ]:
project = 'defect_genome_pcfc_materials'
client = load_client('your-api-key-here')
mpr = MPRester()

Retrieve and update project info

In [ ]:
client.projects.get_entry(pk=project).result()

In [ ]:
client.projects.update_entry(pk=project, project={
    'long_title': 'Defect Genome of Cubic Perovskites for Fuel Cells'
}).result()

Create contributions

In [ ]:
df = read_excel('/Users/patrick/Downloads/DefectGenome_JPCC-data_MP.xlsx')
df.columns = ['A', 'B', 'a', 'Eᶠ|ABO₃', 'Eᶠ|Yᴮ', 'Eᶠ|Vᴼ', 'Eᶠ|Hᵢ', 'ΔEᵢ|Yᴮ−Hᵢ']
units = {'A': '', 'B': '', 'a': 'Å'}
df

In [ ]:
for idx, row in df.iterrows():
    A, B = row[df.columns[0]], row[df.columns[1]]
    formula = f'{A}{B}O3'
    data = mpr.get_data(formula, prop="volume")

    if len(data) > 1:
        volume = row[df.columns[2]]**3
        for d in data:
            d['dV'] = abs(d['volume']-volume)
        data = sorted(data, key=lambda item: item['dV'])
    elif not data:
        print(formula, 'not found on MP')
        continue

    identifier = data[0]['material_id']
    print(idx, formula, identifier)
    
    contrib = {'project': project, 'identifier': identifier, 'data': {}}
    for col in df.columns:
        unit = units.get(col, 'eV')
        contrib['data'][col] = f'{row[col]} {unit}' if unit else row[col]
        
    ntries = 0
    while ntries < 3:
        try:
            for d in client.contributions.get_entries(
                project=project, formula__in=[formula, f'{B}{A}O3'], _fields=['id']
            ).result()['data']:
                client.contributions.delete_entry(pk=d['id']).result()
                #print(d['id'], 'deleted')

            cid = client.contributions.create_entry(contribution=contrib).result()['id']
            #print(client.contributions.get_entry(pk=cid, _fields=['_all']).result())
            #print(cid, 'created')
            break
        except Exception as ex:
            ntries += 1
            print(ntries, ex)
            print('waiting', ntries*30, 'seconds...')
            sleep(ntries*30)
    else:
        print('I give up.')
        break

Bulk update contributions to publish

In [ ]:
client.contributions.update_entries(
    project=project, is_public=False, contributions={'is_public': True}
).result()